Para construir un clasificador multiclase, se puede entrenar 10 clasificadores binarios, uno para cada digito. Luego para cada dígito se devuelve un array de decisiones y se selecciona la clase con el valor mas alto. Resulta que hay una manera mas óptima ya que se puede correr el código una sola vez (a diferencia de una para cada clase) usando **one-hot encoding**.

Ejemplo para un array Y de 4 elementos con 10 clases disponibles (0-9):

$$ \begin{bmatrix} 5 \\ 0 \\ 3 \\ 3 \end{bmatrix} \to \begin{bmatrix} 0&0&0&0&0&1&0&0&0&0 \\ 1&0&0&0&0&0&0&0&0&0 \\ 0&0&0&1&0&0&0&0&0&0 \\ 0&0&0&1&0&0&0&0&0&0 \end{bmatrix}$$ 

Entonces en la etapa de clasificacion, el output del modelo será un array de 10 elementos, donde cada elemento representa la "probabilidad" según el modelo de que el input pertenezca a una clase. La clase con la probabilidad mas alta será la clase seleccionada.

Como ahora hay 10 clases en lugar de 1, tambien hay que actualizar W para que tenga 10 columnas. 
Si antes su dimension era $(785, 1)$ ahora debe ser $(785, 10)$ (*784 pixeles de cada imagen + bias*)

La multiplicacion $XW = Y$ tiene ahora dimension $(m,k)$ con $m$ el numero de filas de X o datos de entrenamiento (60000) y $k$ el numero de columnas de $Y$ o clases (10).

In [21]:
import numpy as np
import gzip
import struct

In [22]:
# importamos las funciones para cargar los datos de entrenamiento de mnist
def load_mnist_images(path):
  with gzip.open(path, 'rb') as f:
		# desempaquetamos: leemos los 4 enteros en big-endian que indican metadatos
    _ignored, n_images, rows, cols = struct.unpack('>IIII', f.read(16))
		# leemos el resto de bytes del archivo como enteros de 8 bits sin signo
    all_pixels = np.frombuffer(f.read(), dtype=np.uint8)
		# reshape para que cada imagen sea una fila (n_images) y cada pixel sea una columna (rows*cols)
    X = all_pixels.reshape(n_images, rows * cols)
  return X

def load_mnist_labels(path):
  with gzip.open(path, 'rb') as f:
		# desempaquetamos: leemos los 2 enteros en big-endian que indican metadatos
    f.read(8)
		# leemos el resto de bytes del archivo como enteros de 8 bits sin signo
    all_labels = f.read()
		# reshape para que cada etiqueta sea una fila
    Y = np.frombuffer(all_labels, dtype=np.uint8).reshape(-1, 1)
  return Y

In [23]:
# definamos la funcion para codificar las etiquetas en one-hot

def one_hot_encode(Y):
	n_labels = Y.shape[0]
	n_classes = 10
	encoded_Y = np.zeros((n_labels, n_classes))

	for i in range(n_labels):
		label = Y[i]
		encoded_Y[i][label] = 1
	
	return encoded_Y

In [24]:
# cargar los datos de entrenamiento y codificar las etiquetas
Y_train = one_hot_encode(load_mnist_labels('../05_logistic_regression/mnist/train-labels-idx1-ubyte.gz'))
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], shape=(60000, 10))

In [25]:
# cargar los datos de inferencia sin codificar las etiquetas
Y_test = load_mnist_labels('../05_logistic_regression/mnist/t10k-labels-idx1-ubyte.gz')
Y_test

array([[7],
       [2],
       [1],
       ...,
       [4],
       [5],
       [6]], shape=(10000, 1), dtype=uint8)

In [26]:
# definir otra vez el modelo
def sigmoid(x) -> float:
	return 1/(1+np.exp(-x))

def forward(X, w) -> np.ndarray:
	weighted_sum = np.matmul(X, w)
	return sigmoid(weighted_sum)

def loss(X, Y, w) -> float:
  y_hat = forward(X, w)
  first_term = Y * np.log(y_hat)
  second_term = (1 - Y) * np.log(1 - y_hat)
  return -np.sum(first_term + second_term) / X.shape[0]

def gradient(X, Y, w) -> np.ndarray:
  return np.matmul(X.T, (forward(X, w) - Y)) / X.shape[0]

In [27]:
# la funcion de clasificacion cambia para seleccionar la clase con mayor probabilidad
def classify(X,w):
	# matriz de predicciones y_hat, una columna para cada clase y una fila para cada dato
	y_hat = forward(X,w)
	# seleccionamos la clase con mayor probabilidad
	# labels es un array de INDICES, el cual coincide con los labels de las clases
	labels = np.argmax(y_hat, axis=1)
	# retornamos las clasificaciones en formato de vector columna
	return labels.reshape(-1,1)


In [28]:
def train(X, Y, learning_rate, iterations):
	w = np.zeros((X.shape[1], Y.shape[1]))

	for i in range(iterations):
		print(f'Iteration {i} => Loss: {loss(X, Y, w)}')
		w -= gradient(X, Y, w) * learning_rate
	
	return w

In [29]:
X_train = np.insert(load_mnist_images('../05_logistic_regression/mnist/train-images-idx3-ubyte.gz'), 0, 1, axis=1)

X_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], shape=(60000, 785), dtype=uint8)

In [31]:
w = train(X_train, Y_train, 1e-5, 1000)

Iteration 0 => Loss: 6.931471805599454
Iteration 1 => Loss: 8.434456875083338
Iteration 2 => Loss: 5.512047488923876
Iteration 3 => Loss: 2.956870073593654
Iteration 4 => Loss: 1.8985387657057093
Iteration 5 => Loss: 1.7558289155266744
Iteration 6 => Loss: 1.674881272926218
Iteration 7 => Loss: 1.623875243420281
Iteration 8 => Loss: 1.5652805689746652
Iteration 9 => Loss: 1.5292692651055577
Iteration 10 => Loss: 1.4834968500183896
Iteration 11 => Loss: 1.4547390723537275
Iteration 12 => Loss: 1.4187844781439438
Iteration 13 => Loss: 1.394256566968422
Iteration 14 => Loss: 1.3659350910622259
Iteration 15 => Loss: 1.3445875188347647
Iteration 16 => Loss: 1.3220198232096096
Iteration 17 => Loss: 1.3034634184193592
Iteration 18 => Loss: 1.285117113766232
Iteration 19 => Loss: 1.2690683151500537
Iteration 20 => Loss: 1.2537827753717958
Iteration 21 => Loss: 1.2398963816638573
Iteration 22 => Loss: 1.2268469039957433
Iteration 23 => Loss: 1.214740575732478
Iteration 24 => Loss: 1.20336782307

In [34]:
# ahora podemos clasificar los datos de inferencia
X_test = np.insert(load_mnist_images('../05_logistic_regression/mnist/t10k-images-idx3-ubyte.gz'), 0, 1, axis=1)

Y_pred = classify(X_test, w)

# evaluamos la precision
accuracy = np.sum(Y_pred == Y_test) / Y_test.shape[0]
accuracy*100

np.float64(91.36999999999999)